In [74]:
import pickle
import pandas as pd
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss
from vit_pytorch import ViT
import tqdm
import numpy as np
import torch
from torch.utils.data import Dataset
import torch
import torchvision.transforms as transforms

In [3]:
# Load the full DataFrame
with open('decomposed_data.pkl', 'rb') as f:
    df = pickle.load(f)

label_map = {'interictal': 0, 'preictal': 1}
df['label'] = df['label'].map(label_map)

In [5]:
# Suppose your DataFrame is df and the tensor column is 'tensor_column'
shapes = df['reduced_spectrogram'].apply(lambda x: x.shape)

# Check if all shapes are the same
all_same_shape = shapes.nunique() == 1

print("All tensors have the same shape:" if all_same_shape else "Tensors have varying shapes.")
print(shapes.value_counts())  # optional: see the distribution of shapes
spectrogram_shape = shapes[0]

All tensors have the same shape:
(119, 65, 10)    7666
Name: reduced_spectrogram, dtype: int64


In [32]:
df['label'].value_counts()

0    6835
1     831
Name: label, dtype: int64

In [40]:
# Separate the two classes
class_0 = df[df['label'] == 0]
class_1 = df[df['label'] == 1]

# Randomly sample from the majority class
class_0_downsampled = class_0.sample(n=len(class_1), random_state=42)

# Combine the downsampled class with the minority class
df_balanced = pd.concat([class_0_downsampled, class_1]).sample(frac=1, random_state=42).reset_index(drop=True)

print("Label counts after balancing:")
print(df_balanced['label'].value_counts())

Label counts after balancing:
1    831
0    831
Name: label, dtype: int64


In [42]:
# Check the first entry
sample_tensor = df_balanced.iloc[0]['reduced_spectrogram']
print("Type:", type(sample_tensor))
print("Shape:", sample_tensor.shape)

Type: <class 'numpy.ndarray'>
Shape: (119, 65, 10)


In [44]:
# Convert and permute the dimensions
tensor = torch.tensor(sample_tensor, dtype=torch.float32)
print("Before permute:", tensor.shape)

tensor = tensor.permute(2, 0, 1)  # from (119, 65, 10) to (10, 119, 65)
print("After permute:", tensor.shape)

Before permute: torch.Size([119, 65, 10])
After permute: torch.Size([10, 119, 65])


In [102]:
import torchvision.transforms as transforms

# Define resize transform
resize_transform = transforms.Compose([
    transforms.Resize((119, 63))  # Resize to dimensions divisible by patch size (7)
])

# Now apply it to the dataset
class SpectrogramDataset(Dataset):
    def __init__(self, df, transform=None):
        self.data = df
        self.transform = transform
        print(f"Initialized dataset with {len(df)} samples")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tensor = self.data.iloc[idx]['reduced_spectrogram']
        label = self.data.iloc[idx]['label']

        # Check if the tensor is an ndarray and convert it to a PyTorch tensor
        if isinstance(tensor, np.ndarray):
            tensor = torch.tensor(tensor, dtype=torch.float32)

        # Permute the tensor to (channels, height, width)
        tensor = tensor.permute(2, 0, 1)
        
        # Apply transformation (e.g., resizing) if provided
        if self.transform:
            tensor = self.transform(tensor)

        # Convert label to tensor
        label = torch.tensor(label, dtype=torch.long)

        #print(f"Sample {idx}: tensor.shape = {tensor.shape}, label = {label.item()}")

        return tensor, label

In [104]:
# Initialize the dataset with resizing transform
dataset = SpectrogramDataset(df_balanced, transform=resize_transform)

# Create DataLoader
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Get a batch and check its shape
for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
    print(f"Batch {batch_idx}: x shape = {x_batch.shape}, y shape = {y_batch.shape}")
    break  # Only look at one batch for now

Initialized dataset with 1662 samples
Batch 0: x shape = torch.Size([16, 10, 119, 63]), y shape = torch.Size([16])


In [106]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VisionTransformer(nn.Module):
    def __init__(self, img_size=(10, 119, 63), patch_size=7, num_classes=2, embed_dim=256, num_heads=8, num_layers=6):
        super(VisionTransformer, self).__init__()
        
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_classes = num_classes
        
        # Calculate number of patches (height, width divided by patch size)
        self.num_patches = (img_size[1] // patch_size) * (img_size[2] // patch_size)
        
        # Patch embedding layer
        self.patch_embed = nn.Conv2d(in_channels=img_size[0], out_channels=embed_dim, kernel_size=patch_size, stride=patch_size)
        
        # Transformer layers
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads),
            num_layers=num_layers
        )
        
        # MLP head for classification
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        # Patch embedding
        x = self.patch_embed(x)
        x = x.flatten(2).transpose(1, 2)  # Shape: (batch_size, num_patches, embed_dim)
        
        # Transformer encoding
        x = self.transformer(x)
        
        # Pooling and classification head
        x = x.mean(dim=1)  # Global average pooling
        x = self.fc(x)
        
        return x

In [ ]:
# Set device to CPU if CUDA is unavailable
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize the model and move it to the device
model = VisionTransformer(img_size=(10, 119, 63), patch_size=7, num_classes=2, embed_dim=256, num_heads=16, num_layers=6)
model = model.to(device)  # Move model to CPU or GPU

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)  # Move data to device
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(x_batch)
        
        # Calculate loss
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
    
    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

Using device: cpu


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch [1/50], Loss: 0.8506, Accuracy: 50.72%
Epoch [2/50], Loss: 0.7302, Accuracy: 49.04%
Epoch [3/50], Loss: 0.7005, Accuracy: 49.52%
Epoch [4/50], Loss: 0.6931, Accuracy: 52.41%
Epoch [5/50], Loss: 0.6943, Accuracy: 49.52%
Epoch [6/50], Loss: 0.6939, Accuracy: 49.94%
Epoch [7/50], Loss: 0.6958, Accuracy: 50.66%
Epoch [8/50], Loss: 0.6961, Accuracy: 49.04%
Epoch [9/50], Loss: 0.6951, Accuracy: 48.80%
Epoch [10/50], Loss: 0.6942, Accuracy: 49.58%
Epoch [11/50], Loss: 0.6937, Accuracy: 49.04%
Epoch [12/50], Loss: 0.6941, Accuracy: 49.28%
Epoch [13/50], Loss: 0.6937, Accuracy: 50.48%
Epoch [14/50], Loss: 0.6953, Accuracy: 48.80%
Epoch [15/50], Loss: 0.6938, Accuracy: 50.84%
Epoch [16/50], Loss: 0.6934, Accuracy: 50.24%
Epoch [17/50], Loss: 0.6948, Accuracy: 50.00%
Epoch [18/50], Loss: 0.6940, Accuracy: 48.68%
